# Выбор локации для скважины

.Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')





data_1= pd.read_csv('/datasets/geo_data_0.csv')
data_2= pd.read_csv('/datasets/geo_data_1.csv')
data_3= pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
print(data_1.head(10))
print(data_1.info())

print(data_2.head(10))
print(data_2.info())

print(data_3.head(10))
print(data_3.info())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
5  wX4Hy  0.969570  0.489775 -0.735383   64.741541
6  tL6pL  0.645075  0.530656  1.780266   49.055285
7  BYPU6 -0.400648  0.808337 -5.624670   72.943292
8  j9Oui  0.643105 -0.551583  2.372141  113.356160
9  OLuZU  2.173381  0.563698  9.441852  127.910945
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.

<b> Вывод </b>:

Импортированы нужные библиотеки, выгружены и изучены данные. Пропущенных значений нет, все данные корректны и готовы для работы.

In [4]:
print(data_1.duplicated().sum())

data_1.describe()

0


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [5]:
print(data_2.duplicated().sum())

data_2.describe()

0


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [6]:
print(data_3.duplicated().sum())

data_3.describe()

0


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


## Обучение и проверка модели

In [7]:
features_1 = data_1.drop(columns=['id', 'product'], axis=1)
target_1 = data_1['product']

features_train_1, features_valid_1 = train_test_split(features_1, test_size=0.25, random_state=12345)
target_train_1, target_valid_1 = train_test_split(target_1, test_size=0.25, random_state=12345)    
 
    
features_2 = data_2.drop(columns=['id', 'product'], axis=1)
target_2 = data_2['product']

features_train_2, features_valid_2 = train_test_split(features_2, test_size=0.25, random_state=12345)
target_train_2, target_valid_2 = train_test_split(target_2, test_size=0.25, random_state=12345)        


features_3 = data_3.drop(columns=['id', 'product'], axis=1)
target_3 = data_3['product']

features_train_3, features_valid_3 = train_test_split(features_3, test_size=0.25, random_state=12345)
target_train_3, target_valid_3 = train_test_split(target_3, test_size=0.25, random_state=12345)  
    
    

<b> Вывод </b>:

Данные разделены на признаки и цели. Цель - столбец product. Признаки - остальные столбцы без целевого столбца и столбца id(уникального идентификатора).

Данные поделены на обучающую и валидационную выборку отдельно для каждого региона в соотношени 75% к 25%

In [26]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predicted_valid_1=model.predict(features_valid_1)
mse = mean_squared_error(target_valid_1, predicted_valid_1)
rmse= mse ** 0.5
mean_1 = predicted_valid_1.mean()
print("RMSE модели первого региона: ", rmse)
print('Средний запас', mean_1)


RMSE модели первого региона:  37.5794217150813
Средний запас 92.59256778438038


In [27]:
predictions = pd.Series(target_valid_1.mean(), index=target_valid_1.index)
mse = mean_squared_error(target_valid_1, predictions)
rmse= mse ** 0.5
print("RMSE:", rmse)

RMSE: 44.28602687855358


In [48]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_valid_2=model_2.predict(features_valid_2)
mse_2 = mean_squared_error(target_valid_2, predicted_valid_2)
rmse_2= mse_2 ** 0.5
mean_2 = predicted_valid_2.mean()
print("RMSE модели второго  региона: ", rmse_2)
print('Средний запас', mean_2)

RMSE модели второго  региона:  0.8930992867756158
Средний запас 68.728546895446


In [18]:
predictions = pd.Series(target_valid_2.mean(), index=target_valid_2.index)
mse = mean_squared_error(target_valid_2, predictions)
rmse= mse ** 0.5
print("RMSE:", rmse)

RMSE: 46.0212449226281


In [29]:
model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)
predicted_valid_3=model_3.predict(features_valid_3)
mse = mean_squared_error(target_valid_3, predicted_valid_3)
rmse= mse ** 0.5
mean_3 = predicted_valid_3.mean()
print("RMSE модели третьего региона: ", rmse)
print('Средний запас', mean_1)

RMSE модели третьего региона:  40.02970873393434
Средний запас 92.59256778438038


In [30]:
predictions = pd.Series(target_valid_3.mean(), index=target_valid_3.index)
mse = mean_squared_error(target_valid_3, predictions)
rmse= mse ** 0.5
print("RMSE:", rmse)

RMSE: 44.902084376898294


In [11]:
results=pd.DataFrame({'Регион':['Первый','Второй','Третий'],
                      'RMSE': ['37.5', '0.89', '40'],
                      'Средний запас предсказанного сырья': ['92.5', '68.7', '92.5'],
                         }) 

print(results)

   Регион  RMSE Средний запас предсказанного сырья
0  Первый  37.5                               92.5
1  Второй  0.89                               68.7
2  Третий    40                               92.5


<b> Вывод </b>:

Обучена модель линейной регрессии для каждого региона, сделаны предсказания, подсчитан средний запас предсказанного сырья и RMSE моделей. Данные собраны в таблицу.

 Для сравнения значений средних ошибок сравнила модели со случайной, добавила в код расчет rmse,  предсказывая всегда среднее значение. Модели адекватны, rmse примерно равно стандартному отклонению в значениях.
Во втором регионе rmse модели сильно расходится  от rmse модели, которая предсказывает всегда среднее число. 
Что говорит о том, что предсказания модели практически не отличаются от правильных.

## Подготовка к расчёту прибыли

In [54]:
point=500
best_point=200
budget= 10000000000
income_product = 450000


budget_one = budget / best_point

V = budget_one / income_product

print('Достаточный объем сырья для безубыточной разработки новой скважины', V)


Достаточный объем сырья для безубыточной разработки новой скважины 111.11111111111111


<b> Вывод </b>:
Все ключевые значения для расчетов сохранены в отдельные переменные. Расчитан достаточный объём сырья для безубыточной разработки новой скважины. Средний запас в каждом регионе меньше значения контрольной точки для безубыточной разработки. Это говорит о том, что запасы в скважине должны быть существенно выше среднего, т.е далеко не все скважины нам подходят.


## Расчёт прибыли и рисков 

In [55]:
#правильная индексация предсказаний целей
predict_sort_1=pd.Series(predicted_valid_1)
target_sort_1=pd.Series(target_valid_1).reset_index(drop=True)

predict_sort_2=pd.Series(predicted_valid_2)
target_sort_2=pd.Series(target_valid_2).reset_index(drop=True)

predict_sort_3=pd.Series(predicted_valid_3)
target_sort_3=pd.Series(target_valid_3).reset_index(drop=True)

#Функция для расчета прибыли по выбранным скважинам и передсказаниям модели

def prodit(target,predict,count):
    predict_max = predict.sort_values(ascending=False)
    target_max = target[predict_max.index][:count]
    return income_product * target_max.sum() - budget

result_1 = prodit(target_sort_1, predict_sort_1, 200)
result_2 = prodit(target_sort_2,predict_sort_2, 200)
result_3 = prodit(target_sort_3,predict_sort_3, 200)

print('Прибыль для полученного объема сырья 1 регион:', result_1)
print('Прибыль для полученного объема сырья 2 регион:', result_2)
print('Прибыль для полученного объема сырья 3 регион:', result_3)

Прибыль для полученного объема сырья 1 регион: 3320826043.1398525
Прибыль для полученного объема сырья 2 регион: 2415086696.681511
Прибыль для полученного объема сырья 3 регион: 2710349963.5998325


<b> Вывод </b>:
Написана функция для расчёта прибыли по выбранным скважинам и предсказаниям модели. Выбрано 200 скважины с максимальными значениями предсказаний. Рассчитана прибыль для полученного объёма сырья

In [56]:
state = np.random.RandomState(12345)

def bootstrapstuff(target,predictions,subsample_size=point):
  
    values = []
    for i in range(1000):
        target_sub = target.sample(n=subsample_size,replace=True,random_state=state)
        predictions_sub = predictions[target_sub.index]
        values.append(prodit(target_sub,predictions_sub,200))
    return values


boot_1 = bootstrapstuff(target_sort_1, predict_sort_1)
values_1= pd.Series(boot_1)
mean_1=values_1.mean()
upper_1 = values_1.quantile(0.975)
lower_1 = values_1.quantile(0.025)
print("Средняя прибыль:", mean_1)
print("97,5%-квантиль:", upper_1)
print("2,5%-квантиль:", lower_1)

Средняя прибыль: 425938526.9105923
97,5%-квантиль: 947976353.358369
2,5%-квантиль: -102090094.83793654


In [57]:
boot_2 = bootstrapstuff(target_sort_2, predict_sort_2)
values_2= pd.Series(boot_2)
mean_2=values_2.mean()
upper_2 = values_2.quantile(0.975)
lower_2 = values_2.quantile(0.025)
print("Средняя прибыль:", mean_2)
print("97,5%-квантиль:", upper_2)
print("2,5%-квантиль:", lower_2)

Средняя прибыль: 518259493.69732493
97,5%-квантиль: 953612982.0669085
2,5%-квантиль: 128123231.43308629


In [58]:
boot_3 = bootstrapstuff(target_sort_3, predict_sort_3)
values_3= pd.Series(boot_3)
mean_3=values_3.mean()
upper_3 = values_3.quantile(0.975)
lower_3 = values_3.quantile(0.025)
print("Средняя прибыль:", mean_3)
print("97,5%-квантиль:", upper_3)
print("2,5%-квантиль:", lower_3)

Средняя прибыль: 420194005.34405005
97,5%-квантиль: 989629939.844574
2,5%-квантиль: -115852609.16001143


<b> Вывод </b>:
Применена техника Bootstrap с 1000 выборок, найдено распределение прибыли для каждого региона, средняя прибыль и риск убытков.


In [131]:
#риск убытков

def risk(values):
    count = 0
    for i in range(len(values)):
            if values[i] < 0:
                count += 1
    return count/ len(values) * 100

print('Риск убытка в 1ом регионе:', risk(values_1), '%')
print('Риск убытка в 2ом регионе:', risk(values_2), '%')
print('Риск убытка в 3ем регионе:', risk(values_3), '%')

Риск убытка в 1ом регионе: 6.0 %
Риск убытка в 2ом регионе: 0.3 %
Риск убытка в 3ем регионе: 6.2 %


In [28]:
results=pd.DataFrame({'Регион':['Первый','Второй','Третий'],
                      'Средняя прибыль(тыс.руб)': ['425', '518', '420'],
                      'Риск убытков': ['6.0 %', '0.3 %', '6.2 %'],
                         }) 

print(results)

   Регион Средняя прибыль(тыс.руб) Риск убытков
0  Первый                      425        6.0 %
1  Второй                      518        0.3 %
2  Третий                      420        6.2 %


<b> Общий Вывод </b>:

В ходе исследования предлагаемый регион для разработки скважин - Второй, так как риск убытков намного ниже, чем в других регионах, а средняя прибыль соответственно выше. Распределение прибыли 95% доверительного интервала - от 128 тысяч до 953 тысяч, при этом как в других регионах 25% квантиль имеет отрицательное значение.